In [1]:
import os 
import pandas as pd
from sklearn import preprocessing
import pickle

In [2]:
filepath = 'C:/Users/kaslou/Desktop/job-recommendation-system/data'

In [3]:
filepath = os.path.join(filepath, 'clean_corpus.csv')

In [4]:
df = pd.read_csv(filepath, delimiter=',')

In [5]:
df.head()

,job_id,account_id,title,description,requirement_summary,benefit_summary,user_keywords,employment_type,function,experience,education,collar_color,soft_skills,technical_skills
0,0,825,project coordinator,open health brings together deep scientific kn...,NaN,NaN,project management research publications growt...,Full-time,Project Management,0.2,Bachelor's Degree,White,attention detail communication organized,publication heor internet research support fol...
1,1,1778,supply chain specialist,droplette revolutionizing drug delivery buildi...,NaN,committed placing much importance betterment e...,NaN,Full-time,Supply Chain,0.3,Bachelor's Degree,White,interdisciplinary detail oriented communicatio...,needle free platform technology material suppl...
2,2,1099,food beverage director,fotografiskaanything ordinary museum fotografi...,average years creating managing food beverage ...,health care plan medical dental vision retirem...,food beverage food beverage management luxury ...,Full-time,Management,0.8,Bachelor's Degree,White,NaN,forecasting fine dining experiences written do...
3,3,1207,quality assurance specialist,acolad international leader language content s...,university degree business management administ...,amplexor offers comprehensive benefits package...,NaN,Full-time,Quality Assurance,0.3,Bachelor's Degree,White,problem solving mindset time management custom...,iso six sigma black belt process improvement b...
4,4,577,people operations associate sep,people operations associate responsible provid...,ideal candidate least year exposure supporting...,return offer competitive salary benefits inclu...,human resources hr people operations,Full-time,Human Resources,0.2,Bachelor's Degree,White,self starter attention detail organizational a...,hr software human hr


In [7]:
columns_to_enumerate = ['employment_type', 'function', 'education', 'collar_color']
for col in columns_to_enumerate:
    le = preprocessing.LabelEncoder()
    df[col] = le.fit_transform(df[col].values)
    
    le.transform()
    
    encoder_filename = 'encode_' + col + '.pickle'
    
    with open(encoder_filename, 'wb') as fout:
        pickle.dump(le, fout, pickle.HIGHEST_PROTOCOL)
    

TypeError: transform() missing 1 required positional argument: 'y'

In [ ]:
list(set(df['education'].values))